```
This colab shows two examples of reading XM measurements.
The first example is for XID and the second one is for XM Measurements.
The second one is more widely used.

Kernel: Brain Frameworks (CPU)
Author: xliucs@
```

In [ ]:
# @title Import & Metrics

from google3.learning.deepmind.xdata import xdata
from google3.learning.deepmind.xmanager2.client import xmanager_api

metric_names = [
    'random_imputation_0.8_eval/valid_mean_squared_error_masked_ignoreimp_mean',
    'imputation_0.00695_eval/valid_mean_squared_error_masked_ignoreimp_mean',
    'imputation_0.02084_eval/valid_mean_squared_error_masked_ignoreimp_mean',
    'imputation_0.04167_eval/valid_mean_squared_error_masked_ignoreimp_mean',
    'imputation_0.125_eval/valid_mean_squared_error_masked_ignoreimp_mean',
    'forecast_0.00695_eval/valid_mean_squared_error_masked_ignoreimp_mean',
    'forecast_0.02084_eval/valid_mean_squared_error_masked_ignoreimp_mean',
    'forecast_0.04167_eval/valid_mean_squared_error_masked_ignoreimp_mean',
    'forecast_0.125_eval/valid_mean_squared_error_masked_ignoreimp_mean',
]

## XData

In [ ]:
def list_to_csv(data_list):
  """Converts a list of strings (representing numbers) to a CSV string.

  Args:
    data_list: A list of strings.

  Returns:
    A string in CSV format, where each element of the list is a cell.
  """
  return ','.join(data_list)


def get_xm_measurement_xdata(xm_id, wid, step, metric_name):
  """Get a XM measurement given a XID and WID"""
  data_id = xdata.get_experiment_data_id(xm_id)

  reader = xdata.bt.reader(data_id, 'data')
  df = reader.read_dataframe()

  xid_df = df[df['XManager/wid'] == wid]
  step_df = xid_df[xid_df['XManager/step'] == step]
  return step_df[metric_name][step_df[metric_name].notna()].values[0]

In [ ]:
xm_id = 159953159
wid = 5
step = 1e5
print('XM ID: ', xm_id)
print('WID: ', wid)
print('Step: ', step)
print('-----------')
values = []
for metric_name in metric_names:
  metric = get_xm_measurement_xdata(xm_id, wid, step, metric_name)
  print(f'{metric_name}: {metric:.2f}')
  values.append(f'{metric:.2f}')

print(list_to_csv(values))

## XMeasurements

In [ ]:
xm_client = xmanager_api.XManagerApi(xm_deployment_env='alphabet')


def read_xm_metrics(example_xid, unit_id, metric_name, lowest=False):
  experiment = xm_client.get_experiment(example_xid)
  work_unit = experiment.get_work_unit(unit_id)
  all_series = work_unit.list_measurement_series()
  # Read measurement series metadata.
  for series in all_series:
    if metric_name in series.label:
      # Read measurement points data.
      all_measurements = []
      for measurement in series.measurements:
        all_measurements.append(measurement.objective_value)
      if lowest:
        return min(all_measurements)
      else:
        return all_measurements[-1]

In [ ]:
xm_id = 156612934
wid = 1
for metric_name in metric_names:
  metric = read_xm_metrics(xm_id, wid, metric_name, lowest=False)
  print(f'{metric_name}: {metric:.2f}')
  values.append(f'{metric:.2f}')
print(list_to_csv(values))